# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.


### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*


In [1]:
import numpy as np
import pandas as pd

def answer_one():
    

    energy = pd.read_excel("Energy Indicators.xls")


    list1 = [x for x in range(0,16)]
    energy = energy.drop(list1)

    list2 = [x for x in range(243,281)]
    energy = energy.drop(list2)

    energy = energy.drop(['Unnamed: 0','Unnamed: 1'], axis = 1)

    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']



    energy['Energy Supply'] = energy['Energy Supply'].replace('...',np.NaN)
    energy['Energy Supply'] *= 1000000


    energy.reset_index(inplace=True, drop=True)




    number_countries = energy[energy['Country'].str.contains('[0-9]+', regex=True)]
    number_values = number_countries['Country'].values
    number_values = list(number_values)
    num_values_dict = {}
    for each_value in number_values:
        res_val = ''.join([i for i in each_value if not i.isdigit()])
        num_values_dict[each_value] = res_val

    energy['Country'].replace(num_values_dict, inplace=True)

    energy['Country'].replace('Republic of Korea', 'South Korea', inplace=True)
    energy['Country'].replace(to_replace='United States of America', value='United States', inplace=True)
    energy['Country'].replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom', inplace=True)
    energy['Country'].replace('China, Hong Kong Special Administrative Region', 'Hong Kong', inplace=True)



    brackets_countries = energy[energy['Country'].str.contains('\(([^)]+)\)', regex=True)]
    brackets_countries = brackets_countries['Country'].values
    brackets_countries = list(brackets_countries)
    brack_count_dict = {}
    for each_value in brackets_countries:
        val = each_value.find('(')
        res_val = each_value[0:val-1]
        brack_count_dict[each_value] = res_val

    energy['Country'].replace(brack_count_dict, inplace=True)

    GDP = pd.read_csv("world_bank.csv")
    GDP = GDP.drop([0,1,2])
    GDP['Data Source'].replace('Korea, Rep.', 'South Korea', inplace=True)
    GDP['Data Source'].replace('Iran, Islamic Rep.', 'Iran', inplace=True)
    GDP['Data Source'].replace('Hong Kong SAR, China', 'Hong Kong', inplace=True)


    ScimEn = pd.read_excel("scimagojr-3.xlsx")
    #ScimEn

    orig_columns = list(GDP.loc[3].values)
    GDP.columns = orig_columns
    GDP = GDP.drop([3])
    GDP = GDP.drop(GDP.columns[4:50], axis=1)
    GDP.reset_index(drop=True,inplace=True)

    GDP.rename(columns = {'Country Name':'Country'}, inplace=True)

    ScimEn.drop([x for x in range(15,191)], inplace=True)

    first_merge_df = pd.merge(energy,GDP,how='inner',left_on='Country',right_on='Country')
    second_merge_df = pd.merge(first_merge_df,ScimEn,how='inner',left_on='Country',right_on='Country')

    second_merge_df.drop(['Country Code','Indicator Name','Indicator Code'],axis=1,inplace=True)
    second_merge_df.set_index('Country', inplace=True)

    second_merge_df.rename(columns = {2006.0:'2006',2007.0:'2007',2008.0:'2008',2009.0:'2009',2010.0:'2010',2011.0:'2011',2012.0:'2012',2013.0:'2013',2014.0:'2014',2015.0:'2015'}, inplace=True)

    second_merge_df = second_merge_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    second_merge_df['% Renewable'] = second_merge_df['% Renewable'].astype('float64')
    second_merge_df['Energy Supply per Capita'] = second_merge_df['Energy Supply per Capita'].astype('float64')
    
    return second_merge_df


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*


In [2]:
#WRONG ANSWER

def answer_two():
    return 305

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*


In [3]:
def answer_three():
    Top15 = answer_one()
    rows = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    third = Top15.apply(lambda x: np.mean(x[rows]), axis=1)

    third = third.sort_values(ascending=False)
    third.name = 'avgGDP'
    return third


### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*


In [4]:
def answer_four():
    Top15 = answer_one()
    rows = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    sixth_largest = Top15.apply(lambda x: np.mean(x[rows]), axis=1)

    sixth_largest = sixth_largest.sort_values(ascending=False)
    coun = sixth_largest.index.tolist()[5]

    final_coun = Top15.loc[coun]
    return (final_coun['2015'] - final_coun['2006'])


### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*


In [5]:
def answer_five():
    Top15 = answer_one()
    return np.mean(Top15['Energy Supply per Capita'])


### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*


In [6]:
def answer_six():
    Top15 = answer_one()
    max_percent = np.max(Top15['% Renewable'])
    max_percent_country = (Top15[Top15['% Renewable'] == max_percent]).index.tolist()[0]
    tuple1 = (max_percent_country, max_percent)
    return tuple1


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*


In [7]:
def answer_seven():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_ratio = np.max(Top15['Ratio'])

    max_ratio_country = (Top15[Top15['Ratio'] == max_ratio]).index.tolist()[0]
    tuple2 = (max_ratio_country, max_ratio)
    return tuple2


### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*


In [8]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    pop = Top15['Population']
    pop = pop.sort_values(ascending=False)
    return pop.index.tolist()[2]


### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*


In [9]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']


    new_df = pd.DataFrame([Top15['Energy Supply per Capita'],Top15['Citable docs per Capita']])

    corr_df = new_df.T.corr(method="pearson")
    return corr_df['Energy Supply per Capita'].values[1]


### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*


In [10]:
def answer_ten():
    Top15 = answer_one()
    medi = np.median(Top15['% Renewable'])
    Top15['HighRenew'] = 1
    Top15_1 = Top15.where(Top15['% Renewable'] >= medi)
    Top15_1 = Top15_1['HighRenew'].fillna(0)

    Top15_1 = Top15_1.astype('int')
    
    Top15_2 = Top15['Rank']

    joined = pd.DataFrame([Top15_1,Top15_2]).T

    ordered_country = joined.sort_values(by='Rank')

    ordered_country = ordered_country['HighRenew']

    ordered_country.name = 'HighRenew'
    
   
    return ordered_country


### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*


In [11]:
def answer_eleven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

    
    ind  = Top15.index.tolist()
    ind
    list1 = []
    for each_ind in ind:
        list1.append(ContinentDict[each_ind])

    Top15['Continent'] = list1
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15.reset_index(inplace=True)
    res = Top15.groupby('Continent').agg({'Country':np.size, 'Population':[np.sum,np.mean,np.std]})
    #Top15.groupby('Continent').agg(size=('Country','size'), sum=('Population','sum'))
    res.columns = res.columns.droplevel()
    return res


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*


In [12]:
def answer_twelve():
    Top15 = answer_one()
    Top15['Bins'] = pd.cut(Top15['% Renewable'], 5)
    Top15['Continent'] = list1
    Top15.reset_index(inplace=True)
    res_df = Top15.groupby(['Continent','Bins']).agg({'Country':np.size})
    res_df.reset_index(inplace=True)

    res_df.set_index(['Continent','Bins'], inplace=True)
    ser_res = res_df['Country']

    return ser_res


### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*


In [13]:
#WRONG ANSWER

def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    popest = Top15['PopEst']
    popest = popest.astype('str')

    total_ele = len(popest.index)
    for j in range(0,total_ele):
        break_list = popest[j].split('.')
        whole_num = break_list[0]
        len1 = len(whole_num)
        list2 = []
        count = 0 
        str2 = ""
        for i in range(len1-1,-1,-1):
            str2 += whole_num[i]
            count += 1
            if (count == 3):
                count = 0
                list2.append(str2)
                str2 = ""

        str2 = ""
        if (count != 0):
            for i in range (count-1,-1,-1):
                str2 += whole_num[i]
            list2.append(str2)

        list2.reverse()

        list3 = []
        for each_ele in list2:
            list3.append(each_ele[::-1])

        whole_num_final = ','.join(list3)
        whole_num_final += "." + break_list[1]

        popest[j] = whole_num_final


    return popest
